In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import os

: 

In [ ]:
# Path directories
train_dir = '../../preprocessing/split/new-dataset-split/train'
val_dir = '../../preprocessing/split/new-dataset-split/val'
test_dir = '../../preprocessing/split/new-dataset-split/test'
output_dir = '/mobilenet'  # Directory to save the model

: 

In [ ]:
# Hyperparameters
image_size = (224, 224)
batch_size = 32
epochs = 10
learning_rate = 0.0001

: 

In [ ]:
# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

: 

In [ ]:
# Load pre-trained MobileNetV2 model without top layer
base_model = MobileNetV2(weights='imagenet', include_top=False)

# Build the model with regularization and dropout
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(x)  # Added L2 regularization
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Added dropout
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

: 

In [ ]:
# Freeze all layers of MobileNetV2
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

: 

In [ ]:
# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

: 

In [ ]:
# Train the model with regularization, dropout, and early stopping
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Fine-tune some layers of the base model
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=learning_rate * 0.1), loss='categorical_crossentropy', metrics=['accuracy'])

: 

In [ ]:
# Fine-tune the model again with regularization, dropout, and early stopping
history_fine_tune = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print(f'Test accuracy: {test_acc}')

: 

In [ ]:
# Save the model in .h5 format
model_path = os.path.join(output_dir, 'model.h5')
model.save(model_path)
print(f'Model saved to: {model_path}')

# Save the model architecture to JSON
model_json = model.to_json()
with open(os.path.join(output_dir, "model.json"), "w") as json_file:
    json_file.write(model_json)

: 

In [ ]:
# Print and save the confusion matrix and classification report
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

: 

In [ ]:
# Get class labels
class_labels = list(test_generator.class_indices.keys())

# Make predictions on the test data
test_generator.reset()
predictions = model.predict(test_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

# True classes
true_classes = test_generator.classes

# Confusion Matrix
cm = confusion_matrix(true_classes, predicted_classes)

: 

In [ ]:
# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.savefig(os.path.join(output_dir, 'confusion_matrix.png'))  # Save the plot
plt.show()

: 

In [ ]:
# Classification Report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

# Print and save precision, recall, and f1-score
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

: 

In [ ]:
# Save model weights to multiple shards
weights = model.get_weights()
num_shards = 4
shard_size = len(weights) // num_shards

: 

In [ ]:
# Save weights to shards
for i in range(num_shards):
    start_index = i * shard_size
    end_index = (i + 1) * shard_size if i < num_shards - 1 else len(weights)
    shard_weights = weights[start_index:end_index]
    shard_file = os.path.join(output_dir, f'group-shard{i+1}of{num_shards}.bin')
    np.save(shard_file, shard_weights)

print(f'Model weights shards saved to: {output_dir}')

: 